| ![EEW logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/eew.jpg?raw=true) | ![EDGI logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/edgi.png?raw=true) |
|---|---|

#### This notebook is licensed under GPL 3.0. Please visit our Github for more information:  https://github.com/edgi-govdata-archiving/
#### The notebook was collaboratively authored by the Environmental Data & Governance Initiative (EDGI) following our authorship protocol: https://docs.google.com/document/d/1CtDN5ZZ4Zv70fHiBTmWkDJ9mswEipX6eCYrwicP66Xw/
#### For more information about this project, visit https://www.environmentalenforcementwatch.org/

## How to Run this Notebook
* If you click on a gray **code** cell, a little “play button” arrow appears on the left. If you click the play button, it will run the code in that cell (“**running** a cell”). The button will animate. When the animation stops, the cell has finished running.
![Where to click to run the cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/pressplay.JPG?raw=true)
* You may get a warning that the notebook was not authored by Google. We know, we authored them! It’s okay. Click “Run Anyway” to continue. 
![Error Message](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/warning-message.JPG?raw=true)
* Run all of the cells in a Notebook to make a complete report. Please feel free to look at and **learn about each result as you create it**!

---

# Analysis of the Climate and Economic Justice Screening Tool

The Biden-Harris administration has long championed a "Justice40" approach to spending climate/environment investments. In early 2021, Biden signed Executive Order XXXXXXX that called on government agencies to prioritize investments in disadvantaged communities. To aid in this effort, the White House Environmental Justiece Advisory Council met to recommend items to be considered in determining what counts as a "disadvatnaged community" (May 2021 final reccommendations report). In February 2022, the Council on Environmental Quality (CEQ) released its draft analysis for the tool, drawing on similar approaches taken by EPA in its EJScreen tool and California's CalEnviroScren (see Github repo for the tool here: XXXXXX).

Here's how the tool works to select "disadvantaged" communities and prioritize them for federal investments. A variety of demographic and environmental indicators were gathered from sources such as the EPA and the Census. These indicators include climate risks (to agriculture), exposure to pollution from traffic, and community-level educational attainment figures. These indicators are assesed at the census tract level, a fairly coarse geographic unit - census tracts comprise block groups, which in turn comprise blocks, the smallest census unit. A tract is identified as disadvantaged if it scores at the 65th percentile or greater on at least three environmental indicators, and at the 80th precentile or greater on one or more demographic indicators. In other words, for each indicator, each census tract is ranked against all others. If it has, for instance, a higher level of wastewater discharge than 65% of all other tracts, then it could qualify as "disdavantaged" (depending on if it met the demographic thresholds). 

In this data science notebook, EDGI's Environmental Enforcement Watch (EEW) presents an analysis of the draft tool, in support of our public comment on it. We believe Justice40 is an important opportunity to address the ongoing legacies of environmental racism. Although the tool does not explicitly consider race, we find (as have others) that it does disproportaiontely identify racialized communities as disadvantaged. 

However, EDGI takes concern with three aspects of the CEJST:
- the tool has no direct indicators of industry compliance with environmental protection laws like the Clean Air Act, nor any measures of state and federal enforcement of these laws. These are fundamental aspects of environmental racism, and hence must be addressed for environmental justice. It is well known that industries act with greater impunity in "disadvantaged" communities that lack political clout (CITE). It is also well known that government agencies at all levels have tended to direct enforcement actions such as penalties and written notices to facilities in wealthier, whiter neighborhoods (CITE). If we are serious about addressing what *causes* higher rates of toxic exposures, climate risks, etc. then we must highlight some of their roots in limited enforcement and compliance. USEPA provides this data (however, here, we have also shown that there are major gaps in EPA's records). A community that has particularly high exposures or risks while also not being served by government seems like a priority for investment and action.
- the tool overemphasizes communities (really, census tracts). It is important to direct federal spending to these communities in just ways. But beyond redressing harm, we must seek to highlight not just communities but structural causes, industries, sectors, driving the problem in the first place. We suggest ways of utilizing USEPA to do this.
- finally, while we are skeptical of an approach reliant on prioritization (of 'scarce' monies that are only artifiically scarce), for very practical purposes we suggest ways CEQ could "prioritize the priorities" so that the most marginalized communities remain in the sights and that Justice40 doesn't get watered down: https://www.eenews.net/articles/congress-earmarks-climate-grants-for-rich-white-areas/

In the analysis that follows, we show how the tool could be adjusted to:
- incorproate indicators of industry compliance and EPA enforcement
- highlight systemic factors *causing* environmental inequalities, instead of *just* redressing harms
- prioritize the priorities

## 1. Get started

In [36]:
# Here we import some extra sources of Python code to help with the analysis
import pandas
import urllib
import numpy
print("Done!")

Done!


In [37]:
# In a separate analysis, available here: https://colab.research.google.com/github/edgi-govdata-archiving/CEJST_Analysis/blob/main/CEJST_prep.ipynb
# we accessed archives of EPA ECHO data through our mirrored copy of it at the Stony Brook University.
# We processed ECHO's historical recorcds to summarize total CWA, CAA, and RCRA violations and inspections per Census tract
# since 2001. We also calculated the violations per facility and inspections per facility in each tract.
echo_data = pandas.read_csv("https://github.com/edgi-govdata-archiving/CEJST_Analysis/raw/main/echo_data.csv", dtype={"FAC_DERIVED_CB2010": str}) # Switch to loading from Github repo # Load as strings to preserve Census tract numbering
echo_data.set_index("FAC_DERIVED_CB2010", inplace=True)
echo_data

,CWA Violations,CAA Violations,RCRA Violations,SDWA Violations,CWA Inspections,CAA Inspections,RCRA Inspections,SDWA Inspections,Facilities,FAC_NAICS_CODES,Total Violations Since 2001,Violations Per Facility Since 2001,Total Inspections Since 2001,Inspections Per Facility Since 2001
FAC_DERIVED_CB2010,,,,,,,,,,,,,,
01001020700,21.0,0.0,5.0,0.0,29.0,31.0,5.0,0.0,2,326112 323111 326111 221320,26.0,13.0,65.0,32.5
01001020801,1.0,0.0,0.0,0.0,32.0,89.0,1.0,11.0,2,221112 22111 322130 325194 32519 322121 325191...,1.0,0.5,133.0,66.5
01001021000,2.0,0.0,0.0,0.0,8.0,82.0,0.0,0.0,1,221112 237990,2.0,2.0,90.0,90.0
01003010300,11.0,0.0,16.0,0.0,11.0,0.0,37.0,0.0,1,325199,27.0,27.0,48.0,48.0
01003010500,51.0,0.0,12.0,0.0,37.0,18.0,39.0,0.0,2,nan 321114,63.0,31.5,94.0,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72151950600,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,2,311311 324191,0.0,0.0,8.0,4.0
72151951000,569.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,1,22132,569.0,569.0,36.0,36.0
72151951300,183.0,0.0,0.0,1.0,17.0,84.0,0.0,0.0,1,324110 81112 32411 324191 424710 493110 492210,184.0,184.0,101.0,101.0


## Bring in CEJST's draft data
Here, we access the CEJST's calculations in order to add our own analysis of government enforcement (inspections) and compliance (quarters in non-compliance) trends.

In [38]:
# Load CEJST data
# Available at https://d3jqyw10j8e7p9.cloudfront.net/data-pipeline/data/score/downloadable/Screening_Tool_Data.zip

import requests, zipfile, io, json

# Load and unzip the data files
url = "https://d3jqyw10j8e7p9.cloudfront.net/data-pipeline/data/score/downloadable/Screening_Tool_Data.zip"
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("/content")

# Add in the CSV spreadsheet
cejst = pandas.read_csv("/content/communities-2022-03-30-0033GMT.csv", dtype={"Census tract ID": str}) # Looks like this filename may change with each download! # We need to load the data as "strings" to avoid 0-leading Census tract IDs from being rendered incorrectly as 1-leading integers.
cejst.set_index("Census tract ID", inplace=True)
cejst

,County Name,State/Territory,Total threshold criteria exceeded,Total categories exceeded,Identified as disadvantaged,Total population,Is low income and has a low percent of higher ed students?,"Greater than or equal to the 90th percentile for expected agriculture loss rate, is low income, and has a low percent of higher ed students?",Expected agricultural loss rate (Natural Hazards Risk Index) (percentile),Expected agricultural loss rate (Natural Hazards Risk Index),...,Percent of individuals below 200% Federal Poverty Line,Percent of individuals < 100% Federal Poverty Line,Percent individuals age 25 or over with less than high school degree (percentile),Percent individuals age 25 or over with less than high school degree,Unemployment (percent) in 2009 (island areas) and 2010 (states and PR),Percentage households below 100% of federal poverty line in 2009 (island areas) and 2010 (states and PR),Greater than or equal to the 90th percentile for unemployment and has low HS education in 2009 (island areas)?,Greater than or equal to the 90th percentile for households at or below 100% federal poverty level and has low HS education in 2009 (island areas)?,Greater than or equal to the 90th percentile for low median household income as a percent of area median income and has low HS education in 2009 (island areas)?,Percent of residents who are not currently enrolled in higher ed
Census tract ID,,,,,,,,,,,,,,,,,,,,,
01001020100,Autauga County,Alabama,0,0,False,1923.0,False,False,45.0,0.0776,...,27.0,16.0,61.0,12.0,2.0,10.0,False,False,False,91.0
01001020200,Autauga County,Alabama,0,0,False,2028.0,True,False,21.0,0.0053,...,43.0,19.0,68.0,14.0,5.0,14.0,False,False,False,96.0
01001020300,Autauga County,Alabama,0,0,False,3476.0,False,False,39.0,0.0527,...,29.0,17.0,68.0,14.0,5.0,8.0,False,False,False,97.0
01001020400,Autauga County,Alabama,0,0,False,3831.0,False,False,43.0,0.0698,...,21.0,4.0,41.0,8.0,3.0,6.0,False,False,False,94.0
01001020500,Autauga County,Alabama,0,0,False,9883.0,False,False,53.0,0.1419,...,20.0,15.0,40.0,7.0,2.0,5.0,False,False,False,94.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78030960900,NaN,Virgin Islands,0,0,False,NaN,False,False,NaN,NaN,...,NaN,NaN,NaN,NaN,10.0,21.0,False,False,False,NaN
78030961000,NaN,Virgin Islands,0,0,False,NaN,False,False,NaN,NaN,...,NaN,NaN,NaN,NaN,10.0,27.0,False,False,False,NaN
78030961100,NaN,Virgin Islands,0,0,False,NaN,False,False,NaN,NaN,...,NaN,NaN,NaN,NaN,7.0,28.0,False,False,False,NaN


In [39]:
# We join the ECHO data to the CEJST data
# Again, the output will be a preview (first and last 5 rows) of the new data.
# You may notice some NaNs in the new columns that were added to the CEJST data, and this is likely because that Census tract has no facilities information in ECHO.
joined = cejst.join(echo_data)
joined[~joined["Violations Per Facility Since 2001"].isna()] # Instead, we can preview the tracts that do have information

,County Name,State/Territory,Total threshold criteria exceeded,Total categories exceeded,Identified as disadvantaged,Total population,Is low income and has a low percent of higher ed students?,"Greater than or equal to the 90th percentile for expected agriculture loss rate, is low income, and has a low percent of higher ed students?",Expected agricultural loss rate (Natural Hazards Risk Index) (percentile),Expected agricultural loss rate (Natural Hazards Risk Index),...,CWA Inspections,CAA Inspections,RCRA Inspections,SDWA Inspections,Facilities,FAC_NAICS_CODES,Total Violations Since 2001,Violations Per Facility Since 2001,Total Inspections Since 2001,Inspections Per Facility Since 2001
Census tract ID,,,,,,,,,,,,,,,,,,,,,
01001020700,Autauga County,Alabama,0,0,False,4029.0,True,False,74.0,0.5200,...,29.0,31.0,5.0,0.0,2.0,326112 323111 326111 221320,26.0,13.0,65.0,32.5
01001020801,Autauga County,Alabama,0,0,False,2826.0,False,False,70.0,0.4140,...,32.0,89.0,1.0,11.0,2.0,221112 22111 322130 325194 32519 322121 325191...,1.0,0.5,133.0,66.5
01001021000,Autauga County,Alabama,0,0,False,2550.0,True,False,72.0,0.4754,...,8.0,82.0,0.0,0.0,1.0,221112 237990,2.0,2.0,90.0,90.0
01003010300,Baldwin County,Alabama,0,0,False,9413.0,False,False,83.0,0.9018,...,11.0,0.0,37.0,0.0,1.0,325199,27.0,27.0,48.0,48.0
01003010500,Baldwin County,Alabama,0,0,False,5444.0,False,False,68.0,0.3713,...,37.0,18.0,39.0,0.0,2.0,nan 321114,63.0,31.5,94.0,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72151950600,Yabucoa Municipio,Puerto Rico,2,1,True,5605.0,True,False,NaN,NaN,...,0.0,8.0,0.0,0.0,2.0,311311 324191,0.0,0.0,8.0,4.0
72151951000,Yabucoa Municipio,Puerto Rico,3,3,True,2833.0,True,False,NaN,NaN,...,36.0,0.0,0.0,0.0,1.0,22132,569.0,569.0,36.0,36.0
72151951300,Yabucoa Municipio,Puerto Rico,3,1,True,5204.0,True,False,NaN,NaN,...,17.0,84.0,0.0,0.0,1.0,324110 81112 32411 324191 424710 493110 492210,184.0,184.0,101.0,101.0


In [40]:
# Now we have the CEJST data, ECHO enforcement/compliance info from EPA, and we're going to add additional Census data on race
# CEJST does not include race information, but much research has shown how important it is to describe environmental inequality.
# We will test how CEJST performs in terms of race - whether it directs monies to communities we know have borne the brunt of environmental hazards.
# We have to manually download/upload the data for this, since existing tools like the Census API can't deliver Census tract info for the entire US
census = pandas.read_csv("https://github.com/edgi-govdata-archiving/CEJST_Analysis/raw/main/DECENNIALSF12010.P5_data_with_overlays_2022-03-30T170415.csv") # Original source: https://data.census.gov/cedsci/table?q=United%20States&t=Race%20and%20Ethnicity&g=0100000US%241400000&y=2010&d=DEC%20Summary%20File%201&tid=DECENNIALSF12010.P5
census = census.drop([0]) # Remove column descriptors
census = census.drop([census.shape[0]]) # Remove last summmary row #census.iloc[74002,]
census["GEO_ID"] = census["GEO_ID"].str.slice(9, 20) # Translate the GEO_ID into the Census tract ID
census = census[["GEO_ID", "P005003"]] # Select the "non-hispanic white population" measure. From this we will calculate the % nonwhite in each tract
census.set_index("GEO_ID", inplace=True)

joined = joined.join(census)
joined["PCT_WHITE"] = (joined["P005003"].astype(numpy.float).astype('Int64')/joined["Total population"].astype(numpy.float).astype('Int64'))*100 # Percent White calculation
joined

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  del sys.path[0]


,County Name,State/Territory,Total threshold criteria exceeded,Total categories exceeded,Identified as disadvantaged,Total population,Is low income and has a low percent of higher ed students?,"Greater than or equal to the 90th percentile for expected agriculture loss rate, is low income, and has a low percent of higher ed students?",Expected agricultural loss rate (Natural Hazards Risk Index) (percentile),Expected agricultural loss rate (Natural Hazards Risk Index),...,RCRA Inspections,SDWA Inspections,Facilities,FAC_NAICS_CODES,Total Violations Since 2001,Violations Per Facility Since 2001,Total Inspections Since 2001,Inspections Per Facility Since 2001,P005003,PCT_WHITE
Census tract ID,,,,,,,,,,,,,,,,,,,,,
01001020100,Autauga County,Alabama,0,0,False,1923.0,False,False,45.0,0.0776,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1601,83.25533
01001020200,Autauga County,Alabama,0,0,False,2028.0,True,False,21.0,0.0053,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,844,41.617357
01001020300,Autauga County,Alabama,0,0,False,3476.0,False,False,39.0,0.0527,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2538,73.01496
01001020400,Autauga County,Alabama,0,0,False,3831.0,False,False,43.0,0.0698,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4030,105.194466
01001020500,Autauga County,Alabama,0,0,False,9883.0,False,False,53.0,0.1419,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8438,85.378934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78030960900,NaN,Virgin Islands,0,0,False,NaN,False,False,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
78030961000,NaN,Virgin Islands,0,0,False,NaN,False,False,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
78030961100,NaN,Virgin Islands,0,0,False,NaN,False,False,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>


In [41]:
# Some basic diagnostics
#joined[joined['Violations Per Facility Since 2001'].isna()] #59973 tracts without violations
#joined[joined['Inspections Per Facility Since 2001'].isna()] #59973 tracts without inspections
#joined[joined['Facilities'].isna()] #59973 tracts without major facilities
joined[joined['Facilities']>0] #14161 tracts with major facilities

,County Name,State/Territory,Total threshold criteria exceeded,Total categories exceeded,Identified as disadvantaged,Total population,Is low income and has a low percent of higher ed students?,"Greater than or equal to the 90th percentile for expected agriculture loss rate, is low income, and has a low percent of higher ed students?",Expected agricultural loss rate (Natural Hazards Risk Index) (percentile),Expected agricultural loss rate (Natural Hazards Risk Index),...,RCRA Inspections,SDWA Inspections,Facilities,FAC_NAICS_CODES,Total Violations Since 2001,Violations Per Facility Since 2001,Total Inspections Since 2001,Inspections Per Facility Since 2001,P005003,PCT_WHITE
Census tract ID,,,,,,,,,,,,,,,,,,,,,
01001020700,Autauga County,Alabama,0,0,False,4029.0,True,False,74.0,0.5200,...,5.0,0.0,2.0,326112 323111 326111 221320,26.0,13.0,65.0,32.5,2294,56.937205
01001020801,Autauga County,Alabama,0,0,False,2826.0,False,False,70.0,0.4140,...,1.0,11.0,2.0,221112 22111 322130 325194 32519 322121 325191...,1.0,0.5,133.0,66.5,2660,94.125973
01001021000,Autauga County,Alabama,0,0,False,2550.0,True,False,72.0,0.4754,...,0.0,0.0,1.0,221112 237990,2.0,2.0,90.0,90.0,2208,86.588235
01003010300,Baldwin County,Alabama,0,0,False,9413.0,False,False,83.0,0.9018,...,37.0,0.0,1.0,325199,27.0,27.0,48.0,48.0,6038,64.145331
01003010500,Baldwin County,Alabama,0,0,False,5444.0,False,False,68.0,0.3713,...,39.0,0.0,2.0,nan 321114,63.0,31.5,94.0,47.0,3885,71.362968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72151950600,Yabucoa Municipio,Puerto Rico,2,1,True,5605.0,True,False,NaN,NaN,...,0.0,0.0,2.0,311311 324191,0.0,0.0,8.0,4.0,58,1.03479
72151951000,Yabucoa Municipio,Puerto Rico,3,3,True,2833.0,True,False,NaN,NaN,...,0.0,0.0,1.0,22132,569.0,569.0,36.0,36.0,9,0.317684
72151951300,Yabucoa Municipio,Puerto Rico,3,1,True,5204.0,True,False,NaN,NaN,...,0.0,0.0,1.0,324110 81112 32411 324191 424710 493110 492210,184.0,184.0,101.0,101.0,33,0.634128


In [42]:
# We will create "scores" - percentage ranks to be specific - for our enforcement and compliance indicators.
# Just like CEJST, we will assign each Census tract percentage ranks. 
# We will do so for "inspections per facility in this tract since 2001" 
# and "violations per facility in this tract since 2001"

# Calculate the scores
joined['Violations_PR'] = joined["Violations Per Facility Since 2001"].rank(pct = True)
joined['Inspections_PR'] = joined["Inspections Per Facility Since 2001"].rank(pct = True)
joined[~joined["Inspections_PR"].isna()] # Show only tracts withs cores

,County Name,State/Territory,Total threshold criteria exceeded,Total categories exceeded,Identified as disadvantaged,Total population,Is low income and has a low percent of higher ed students?,"Greater than or equal to the 90th percentile for expected agriculture loss rate, is low income, and has a low percent of higher ed students?",Expected agricultural loss rate (Natural Hazards Risk Index) (percentile),Expected agricultural loss rate (Natural Hazards Risk Index),...,Facilities,FAC_NAICS_CODES,Total Violations Since 2001,Violations Per Facility Since 2001,Total Inspections Since 2001,Inspections Per Facility Since 2001,P005003,PCT_WHITE,Violations_PR,Inspections_PR
Census tract ID,,,,,,,,,,,,,,,,,,,,,
01001020700,Autauga County,Alabama,0,0,False,4029.0,True,False,74.0,0.5200,...,2.0,326112 323111 326111 221320,26.0,13.0,65.0,32.5,2294,56.937205,0.610585,0.629263
01001020801,Autauga County,Alabama,0,0,False,2826.0,False,False,70.0,0.4140,...,2.0,221112 22111 322130 325194 32519 322121 325191...,1.0,0.5,133.0,66.5,2660,94.125973,0.190947,0.866535
01001021000,Autauga County,Alabama,0,0,False,2550.0,True,False,72.0,0.4754,...,1.0,221112 237990,2.0,2.0,90.0,90.0,2208,86.588235,0.310818,0.919638
01003010300,Baldwin County,Alabama,0,0,False,9413.0,False,False,83.0,0.9018,...,1.0,325199,27.0,27.0,48.0,48.0,6038,64.145331,0.737624,0.773851
01003010500,Baldwin County,Alabama,0,0,False,5444.0,False,False,68.0,0.3713,...,2.0,nan 321114,63.0,31.5,94.0,47.0,3885,71.362968,0.763541,0.766012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72151950600,Yabucoa Municipio,Puerto Rico,2,1,True,5605.0,True,False,NaN,NaN,...,2.0,311311 324191,0.0,0.0,8.0,4.0,58,1.03479,0.090354,0.088765
72151951000,Yabucoa Municipio,Puerto Rico,3,3,True,2833.0,True,False,NaN,NaN,...,1.0,22132,569.0,569.0,36.0,36.0,9,0.317684,0.995940,0.669232
72151951300,Yabucoa Municipio,Puerto Rico,3,1,True,5204.0,True,False,NaN,NaN,...,1.0,324110 81112 32411 324191 424710 493110 492210,184.0,184.0,101.0,101.0,33,0.634128,0.968858,0.936410


## *Grow* CJEST's existing list based on enforcement/compliance metrics

In [43]:
# Generally, here's how CEJST currently selects "disadvantaged" areas":
# Percent higher ed enrollment rate is less than 20% *AND* a significance share of the population qualifies as low income (65th percentile nationally)
# AND the tract scores higher than the 90th percentile in at least one environmental indicator (wastewater discharge, traffic pollution, expected agricultural losses due to climate change)
# There are some exceptions to this (e.g. workforce development). 
# For our purposes, we just use the "Identified as disadvantaged" field to report on what CEJST has already id'd.

# We say an area could be considered "disadvantaged" if it has a higher ed enrollment rate < 20% and significant low income population, 
# AND is in the 90th percentile on either enforcement/compliance measure (high violations per facility since 2001, low inspections per facility since 2001)
not_da = joined.loc[joined['Identified as disadvantaged']==False] # Areas not already id'd as disadvantaged
not_da = not_da.loc[not_da['Is low income and has a low percent of higher ed students?'] == True] 
potentials = not_da.loc[(not_da["Inspections_PR"] < .1) | (not_da["Violations_PR"] > .9)] # Areas not already id'd as disadvantaged, but our enforcement/compliance index and other factors suggest it could be
potentials

,County Name,State/Territory,Total threshold criteria exceeded,Total categories exceeded,Identified as disadvantaged,Total population,Is low income and has a low percent of higher ed students?,"Greater than or equal to the 90th percentile for expected agriculture loss rate, is low income, and has a low percent of higher ed students?",Expected agricultural loss rate (Natural Hazards Risk Index) (percentile),Expected agricultural loss rate (Natural Hazards Risk Index),...,Facilities,FAC_NAICS_CODES,Total Violations Since 2001,Violations Per Facility Since 2001,Total Inspections Since 2001,Inspections Per Facility Since 2001,P005003,PCT_WHITE,Violations_PR,Inspections_PR
Census tract ID,,,,,,,,,,,,,,,,,,,,,
01127020200,Walker County,Alabama,0,0,False,4561.0,True,False,72.0,0.4653,...,2.0,311119 321113,0.0,0.000000,3.0,1.500000,3820,83.753563,0.090354,0.055610
04013092500,Maricopa County,Arizona,0,0,False,4915.0,True,False,17.0,0.0030,...,1.0,NaN,0.0,0.000000,1.0,1.000000,1840,37.436419,0.090354,0.046748
04015952600,Mohave County,Arizona,0,0,False,3094.0,True,False,14.0,0.0016,...,1.0,221112,1.0,1.000000,2.0,2.000000,2359,76.244344,0.237307,0.062143
05001480500,Arkansas County,Arkansas,0,0,False,3928.0,True,False,66.0,0.3374,...,1.0,NaN,0.0,0.000000,0.0,0.000000,3085,78.538697,0.090354,0.018749
05007021002,Benton County,Arkansas,0,0,False,7492.0,True,False,74.0,0.5342,...,3.0,221112 221119 237990 337110 33711 221320,479.0,159.666667,91.0,30.333333,5241,69.954618,0.960455,0.600946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53071920200,Walla Walla County,Washington,0,0,False,4416.0,True,False,0.0,0.0000,...,1.0,22132,159.0,159.000000,16.0,16.000000,2683,60.756341,0.960314,0.320175
54051020900,Marshall County,West Virginia,0,0,False,4853.0,True,False,39.0,0.0507,...,10.0,32518 325180 325181 325188 325110 1 325192 221...,1104.0,110.400000,356.0,35.600000,5189,106.923552,0.931008,0.664360
56015957900,Goshen County,Wyoming,0,0,False,5107.0,True,False,82.0,0.8312,...,1.0,NaN,84.0,84.000000,31.0,31.000000,4278,83.767378,0.903149,0.609314


In [44]:
# We describe some aspects of these potential CEJST target tracts
sum = potentials['Total population'].astype(int).sum()
display("Total population in these areas: {:,}".format(sum))

states = potentials.groupby(by="State/Territory")[["State/Territory"]].agg("count")
display("Number of tracts per state/territory: ")
display(states)

'Total population in these areas: 537,584'

'Number of tracts per state/territory: '

,State/Territory
State/Territory,
Alabama,1
Arizona,2
Arkansas,2
California,3
Connecticut,1
Florida,5
Georgia,15
Hawaii,1
Idaho,3


In [45]:
# A more detailed comparison of these potential targets with tracts already identified and those not disadvantaged
idd = joined[joined['Identified as disadvantaged']==True] # Tracts already identified as disdvantaged
nots = joined[(joined['Identified as disadvantaged']==False) & (~(joined.index.isin(list(potentials.index))))] # Tracts not identified as disdvantaged and not targets we have identified

# Calculate the median values for different CEJST indicators
# Report on how these vary across all census tracts, already identified disadvantaged areas, the ones we have identified in our analysis, and not disdavantaged areas
results = []
for c in joined.columns:
  if (joined[c].dtype != bool) & (joined[c].dtype != object): # Skip columns where values are just true/false
    j = round(joined[c].median(), 2) # Round numbers to 2 decimal places
    i = round(idd[c].median(), 2)
    n = round(nots[c].median(), 2)
    p = round(potentials[c].median(),2 )
    results.append({"indicator": c, "All Tracts": j, "Disadvantaged": i, "Not Disadvantaged": n, "Potentially Disadvantaged": p})
results = pandas.DataFrame.from_records(results)
results.set_index("indicator", inplace=True)

for indicator in list(results.index.unique()):
  display(results[results.index == indicator])

,All Tracts,Disadvantaged,Not Disadvantaged,Potentially Disadvantaged
indicator,,,,
Total threshold criteria exceeded,0.0,3.0,0.0,0.0


,All Tracts,Disadvantaged,Not Disadvantaged,Potentially Disadvantaged
indicator,,,,
Total categories exceeded,0.0,2.0,0.0,0.0


,All Tracts,Disadvantaged,Not Disadvantaged,Potentially Disadvantaged
indicator,,,,
Total population,4113.0,3735.0,4287.0,4877.0


,All Tracts,Disadvantaged,Not Disadvantaged,Potentially Disadvantaged
indicator,,,,
Expected agricultural loss rate (Natural Hazards Risk Index) (percentile),27.0,9.0,33.0,59.0


,All Tracts,Disadvantaged,Not Disadvantaged,Potentially Disadvantaged
indicator,,,,
Expected agricultural loss rate (Natural Hazards Risk Index),0.01,0.0,0.03,0.2


...

,All Tracts,Disadvantaged,Not Disadvantaged,Potentially Disadvantaged
indicator,,,,
Total Inspections Since 2001,32.0,34.0,32.0,14.0


,All Tracts,Disadvantaged,Not Disadvantaged,Potentially Disadvantaged
indicator,,,,
Inspections Per Facility Since 2001,24.0,24.75,24.25,13.0


,All Tracts,Disadvantaged,Not Disadvantaged,Potentially Disadvantaged
indicator,,,,
PCT_WHITE,69.56,32.44,77.49,73.26


,All Tracts,Disadvantaged,Not Disadvantaged,Potentially Disadvantaged
indicator,,,,
Violations_PR,0.5,0.5,0.5,0.91


,All Tracts,Disadvantaged,Not Disadvantaged,Potentially Disadvantaged
indicator,,,,
Inspections_PR,0.49,0.5,0.5,0.25


In [46]:
# Show where there are significant differences between our approach and CEJST

differences = results.copy() # Make a copy of the results

# Calculate percent differences
for indicator in list(differences.index.unique()):
  differences.loc[differences.index == indicator, "All Tracts"] = round((differences.loc[differences.index == indicator, "Potentially Disadvantaged"] / 
                                                                   differences.loc[differences.index == indicator, "All Tracts"]) * 100, 2)
  differences.loc[differences.index == indicator, "Disadvantaged"] = round((differences.loc[differences.index == indicator, "Potentially Disadvantaged"] / 
                                                                      differences.loc[differences.index == indicator, "Disadvantaged"]) * 100, 2)
  differences.loc[differences.index == indicator, "Not Disadvantaged"] = round((differences.loc[differences.index == indicator, "Potentially Disadvantaged"] / 
                                                                          differences.loc[differences.index == indicator, "Not Disadvantaged"]) * 100, 2)
  differences.loc[differences.index == indicator, "Potentially Disadvantaged"] = round((differences.loc[differences.index == indicator, "Potentially Disadvantaged"] / 
                                                                          differences.loc[differences.index == indicator, "Potentially Disadvantaged"]) * 100, 2)
differences

,All Tracts,Disadvantaged,Not Disadvantaged,Potentially Disadvantaged
indicator,,,,
Total threshold criteria exceeded,NaN,0.00,NaN,NaN
Total categories exceeded,NaN,0.00,NaN,NaN
Total population,118.58,130.58,113.76,100.0
Expected agricultural loss rate (Natural Hazards Risk Index) (percentile),218.52,655.56,178.79,100.0
Expected agricultural loss rate (Natural Hazards Risk Index),2000.00,inf,666.67,100.0
...,...,...,...,...
Total Inspections Since 2001,43.75,41.18,43.75,100.0
Inspections Per Facility Since 2001,54.17,52.53,53.61,100.0
PCT_WHITE,105.32,225.83,94.54,100.0


In [47]:
# Output shows how much greater or less than the "potentially disadvantaged" median is for each indicator in relation to the other kinds of tracts
# >100 = Potentially disadvantaged areas have a *higher* median (e.g. a higher median population) than all US Census tracts or disadvantaged tracts or not disadvantaged tracts
# <100 = Potentially disadvantaged areas have a *lower* median (e.g. a lower median percent white population)
# NaNs and infs represent situations with 0s as denominators i.e. indicators for which the potentially disdvantaged areas have 0s as the median

# We'll first focus on indicators where potentially disadvantaged areas have higher medians than already identified disadvantaged areas
differences[differences["Disadvantaged"] > 100]

,All Tracts,Disadvantaged,Not Disadvantaged,Potentially Disadvantaged
indicator,,,,
Total population,118.58,130.58,113.76,100.0
Expected agricultural loss rate (Natural Hazards Risk Index) (percentile),218.52,655.56,178.79,100.0
Expected agricultural loss rate (Natural Hazards Risk Index),2000.00,inf,666.67,100.0
Expected building loss rate (Natural Hazards Risk Index) (percentile),126.00,128.57,126.00,100.0
Expected building loss rate (Natural Hazards Risk Index),200.00,200.00,200.00,100.0
Expected population loss rate (Natural Hazards Risk Index) (percentile),128.00,128.00,130.61,100.0
Median value ($) of owner-occupied housing units (percentile),52.00,118.18,44.07,100.0
Median value ($) of owner-occupied housing units,64.89,109.11,53.38,100.0
Life expectancy (years),97.07,100.93,95.74,100.0


In [48]:
# Indicators where our "potentially disadvantaged areas" have lower median scores than already identified disadvantaged areas
differences[differences["Disadvantaged"] < 100 ]

,All Tracts,Disadvantaged,Not Disadvantaged,Potentially Disadvantaged
indicator,,,,
Total threshold criteria exceeded,NaN,0.00,NaN,NaN
Total categories exceeded,NaN,0.00,NaN,NaN
Energy burden (percentile),134.69,88.00,173.68,100.0
Energy burden,150.00,75.00,150.00,100.0
PM2.5 in the air (percentile),99.00,88.39,105.32,100.0
PM2.5 in the air,99.88,97.93,100.83,100.0
Diesel particulate matter exposure (percentile),64.00,55.17,69.57,100.0
Diesel particulate matter exposure,66.67,56.52,72.22,100.0
Traffic proximity and volume (percentile),74.00,62.71,82.22,100.0


## Examine "disdvantaged areas" in terms of sectors

In [59]:
# We'll return to the EPA ECHO data to see if we can determine some commonalities in what CEJST has identified as "disadvantaged areas"
# We'll look at NAICS codes for these facilities to see if there are common industries
# First, we'll get the Census tracts id'd as "disadvantaged"
tracts = list(idd.index.unique())
# Look up the disadvantaged tracts
facs_in_tracts = echo_data.loc[echo_data.index.isin(tracts)] ## 4813 tracts out of 23,000 already identified as disadvantaged have major facilities
facs_in_tracts['Facilities'].sum() # 8350 or about 33% of the 25,000 major facilities EPA monitors, are in census tracts CEJST identified as disdavantaged
facs_in_tracts

,CWA Violations,CAA Violations,RCRA Violations,SDWA Violations,CWA Inspections,CAA Inspections,RCRA Inspections,SDWA Inspections,Facilities,FAC_NAICS_CODES,Total Violations Since 2001,Violations Per Facility Since 2001,Total Inspections Since 2001,Inspections Per Facility Since 2001
FAC_DERIVED_CB2010,,,,,,,,,,,,,,
01005950200,0.0,0.0,0.0,0.0,1.0,6.0,0.0,0.0,2,321219 321219,0.0,0.0,7.0,3.5
01005950300,1.0,0.0,1.0,0.0,9.0,14.0,3.0,0.0,1,321114 321912,2.0,2.0,26.0,26.0
01005950500,0.0,0.0,0.0,0.0,0.0,46.0,0.0,0.0,1,327999 327120,0.0,0.0,46.0,46.0
01005950600,0.0,1.0,0.0,0.0,3.0,46.0,0.0,0.0,1,212324 236210 327124,1.0,1.0,49.0,49.0
01005950700,27.0,2.0,0.0,0.0,2.0,48.0,0.0,0.0,1,321113 321999,29.0,29.0,50.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72151950600,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,2,311311 324191,0.0,0.0,8.0,4.0
72151951000,569.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,1,22132,569.0,569.0,36.0,36.0
72151951300,183.0,0.0,0.0,1.0,17.0,84.0,0.0,0.0,1,324110 81112 32411 324191 424710 493110 492210,184.0,184.0,101.0,101.0


In [50]:
# There are many factors that CEJST uses to determine whether a tract is "disadvantaged" and not all of them are driven by industrial facilities
# For instance, traffic pollution, expected agricultural losses, and workforce development scores can each make a tract "count" as disadvantaged
# But they aren't things driven by industrial polluters EPA regulates and reports on in ECHO

# That said, many of the CEJST categories that count, even ones that don't seem connected to facilities, like asthma rates, are very relevant
# Let's look more closely at the NAICS codes for facilities in identified tracts
# Look up the codes in a NAICS lookup table to determine industries
# We will load in a file that tells us about the type of industry facilities are classified as
codes = pandas.read_excel("https://www.census.gov/naics/2017NAICS/6-digit_2017_Codes.xlsx",
  header = 0, # the row that represents the header
  converters={'2017 NAICS Code':str,'2017 NAICS Title':str}
)
codes = codes[["2017 NAICS Code", "2017 NAICS Title"]]
codes["# of Facilities"] = 0
codes["# of Tracts"] = 0

tractwise=dict((n,[]) for n in list(codes['2017 NAICS Code'].unique())) # For counting the number of tracts that a facility code falls within

# Now we will replace each facility's NAICS code with its NAICS title
# This may take some time!
def counter(row):
  n = row["FAC_NAICS_CODES"]
  n = str(n).split(" ")
  for c in n:
    codes.loc[codes["2017 NAICS Code"] == c, "# of Facilities"] += 1 # Is the code in the NAICS lookup? If so, count it
    if codes.loc[codes["2017 NAICS Code"] == c].shape[0] > 0: # Is the code in the NAICS lookup? Record it as in this tract
      tractwise[c].append(row.name) #["FAC_DERIVED_CB2010"]
  #display(codes.loc[codes["2017 NAICS Code"].isin(n)]) # Debugging
facs_in_tracts["NAICS_Titles"] = echo_data.apply (lambda row: counter(row), axis=1)

for code, ts in tractwise.items():
  count = len(set(ts)) #[ (i,tracts.count(i)) for i in set(tracts) ]
  codes.loc[codes['2017 NAICS Code'] == code, "# of Tracts"] = count

codes

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,2017 NAICS Code,2017 NAICS Title,# of Facilities,# of Tracts
0,NaN,NaN,0,0
1,111110,Soybean Farming,5,5
2,111120,Oilseed (except Soybean) Farming,2,2
3,111130,Dry Pea and Bean Farming,0,0
4,111140,Wheat Farming,1,1
...,...,...,...,...
1053,926140,Regulation of Agricultural Marketing and Commo...,2,2
1054,926150,"Regulation, Licensing, and Inspection of Misce...",0,0
1055,927110,Space Research and Technology,13,13
1056,928110,National Security,222,206


In [51]:
# Note that the NAICS numbers will add up to more than the number of facilities in disadvantaged tracts
# A facility can have, and often does have, multiple NAICS codes associated with it
# Let's sort and rank these. First, in termsere are the top 20 industry categories:
codes.sort_values(by=["# of Facilities"], ascending=False).head(20)

,2017 NAICS Code,2017 NAICS Title,# of Facilities,# of Tracts
94,221112,Fossil Fuel Electric Power Generation,2049,1766
105,221320,Sewage Treatment Facilities,1509,1462
666,486210,Pipeline Transportation of Natural Gas,1312,993
877,562212,Solid Waste Landfill,1009,964
284,326199,All Other Plastics Product Manufacturing,633,569
251,325199,All Other Basic Organic Chemical Manufacturing,530,360
66,211130,Natural Gas Extraction,470,295
557,424710,Petroleum Bulk Stations and Terminals,438,327
876,562211,Hazardous Waste Treatment and Disposal,436,404
252,325211,Plastics Material and Resin Manufacturing,426,314


In [52]:
# Then we'll sort it by # of "disadvantaged" tracts that are home to a kind of industry
codes.sort_values(by=["# of Tracts"], ascending=False).head(20)

,2017 NAICS Code,2017 NAICS Title,# of Facilities,# of Tracts
94,221112,Fossil Fuel Electric Power Generation,2049,1766
105,221320,Sewage Treatment Facilities,1509,1462
666,486210,Pipeline Transportation of Natural Gas,1312,993
877,562212,Solid Waste Landfill,1009,964
284,326199,All Other Plastics Product Manufacturing,633,569
876,562211,Hazardous Waste Treatment and Disposal,436,404
102,221122,Electric Power Distribution,416,392
251,325199,All Other Basic Organic Chemical Manufacturing,530,360
557,424710,Petroleum Bulk Stations and Terminals,438,327
252,325211,Plastics Material and Resin Manufacturing,426,314


## Prioritize the priorities - *shrink* CJEST's existing list based on enforcement/compliance metrics

In [53]:
# Here we look at *shrinking* the existing list based on our enforcement/compliance metrics
priorities = idd.loc[(idd["Inspections_PR"] < .1) | (idd["Violations_PR"] > .9)]
priorities

,County Name,State/Territory,Total threshold criteria exceeded,Total categories exceeded,Identified as disadvantaged,Total population,Is low income and has a low percent of higher ed students?,"Greater than or equal to the 90th percentile for expected agriculture loss rate, is low income, and has a low percent of higher ed students?",Expected agricultural loss rate (Natural Hazards Risk Index) (percentile),Expected agricultural loss rate (Natural Hazards Risk Index),...,Facilities,FAC_NAICS_CODES,Total Violations Since 2001,Violations Per Facility Since 2001,Total Inspections Since 2001,Inspections Per Facility Since 2001,P005003,PCT_WHITE,Violations_PR,Inspections_PR
Census tract ID,,,,,,,,,,,,,,,,,,,,,
01005950200,Barbour County,Alabama,3,2,True,3795.0,True,False,89.0,1.4859,...,2.0,321219 321219,0.0,0.0,7.0,3.5,1457,38.392622,0.090354,0.081809
01013953200,Butler County,Alabama,2,2,True,3385.0,True,True,91.0,1.6885,...,1.0,NaN,95.0,95.0,28.0,28.0,2604,76.927622,0.917202,0.562531
01015002300,Calhoun County,Alabama,5,4,True,3622.0,True,False,70.0,0.4103,...,1.0,221320,193.0,193.0,27.0,27.0,3214,88.735505,0.971153,0.545159
01031011300,Coffee County,Alabama,5,3,True,3885.0,True,True,94.0,2.5985,...,2.0,332999 33211 332312 nan,183.0,91.5,72.0,36.0,1822,46.898327,0.912647,0.669232
01037961000,Coosa County,Alabama,2,2,True,3705.0,True,False,82.0,0.8420,...,1.0,339995,0.0,0.0,3.0,3.0,1537,41.48448,0.090354,0.075454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72145560300,Vega Baja Municipio,Puerto Rico,3,2,True,3263.0,True,False,NaN,NaN,...,1.0,22132,1467.0,1467.0,64.0,64.0,19,0.582286,0.999647,0.857814
72151950600,Yabucoa Municipio,Puerto Rico,2,1,True,5605.0,True,False,NaN,NaN,...,2.0,311311 324191,0.0,0.0,8.0,4.0,58,1.03479,0.090354,0.088765
72151951000,Yabucoa Municipio,Puerto Rico,3,3,True,2833.0,True,False,NaN,NaN,...,1.0,22132,569.0,569.0,36.0,36.0,9,0.317684,0.995940,0.669232


In [54]:
# We describe some aspects of these *prioritized* CEJST target tracts
# Out of the 23,000 already identified, there are 1025 that could be priorities based on enforcement/compliance metrics
sum = priorities['Total population'].sum()
display("Total population in 'priority' areas: {:,}".format(sum))

states = priorities.groupby(by="State/Territory")[["State/Territory"]].agg("count")
display("Number of tracts per state/territory: ")
display(states)

"Total population in 'priority' areas: 4,218,258.0"

'Number of tracts per state/territory: '

,State/Territory
State/Territory,
Alabama,26
American Samoa,2
Arizona,13
Arkansas,26
California,30
Colorado,6
Connecticut,17
District of Columbia,2
Florida,83


In [55]:
# Calculate the median values for different CEJST indicators
# Report on how these vary across all census tracts, OTHER already identified disadvantaged areas, not disdavantaged areas, and "priorities"
not_priorities = joined[(joined['Identified as disadvantaged']==True) & (~(joined.index.isin(list(priorities.index))))] # Tracts identified as disdvantaged and not targets we have identified as priorities
results = []
for c in joined.columns:
  if (joined[c].dtype != bool) & (joined[c].dtype != object): # Skip columns where values are just true/false
    j = round(joined[c].median(), 2) # Round numbers to 2 decimal places
    i = round(not_priorities[c].median(), 2)
    n = round(nots[c].median(), 2)
    p = round(priorities[c].median(),2 )
    results.append({"indicator": c, "All Tracts": j, "Disadvantaged but Not Priorities": i, "Not Disadvantaged": n, "Priorities": p})
results = pandas.DataFrame.from_records(results)
results.set_index("indicator", inplace=True)

for indicator in list(results.index.unique()):
  display(results[results.index == indicator])

,All Tracts,Disadvantaged but Not Priorities,Not Disadvantaged,Priorities
indicator,,,,
Total threshold criteria exceeded,0.0,3.0,0.0,3.0


,All Tracts,Disadvantaged but Not Priorities,Not Disadvantaged,Priorities
indicator,,,,
Total categories exceeded,0.0,2.0,0.0,2.0


,All Tracts,Disadvantaged but Not Priorities,Not Disadvantaged,Priorities
indicator,,,,
Total population,4113.0,3732.5,4287.0,3804.0


,All Tracts,Disadvantaged but Not Priorities,Not Disadvantaged,Priorities
indicator,,,,
Expected agricultural loss rate (Natural Hazards Risk Index) (percentile),27.0,7.0,33.0,44.0


,All Tracts,Disadvantaged but Not Priorities,Not Disadvantaged,Priorities
indicator,,,,
Expected agricultural loss rate (Natural Hazards Risk Index),0.01,0.0,0.03,0.07


...

,All Tracts,Disadvantaged but Not Priorities,Not Disadvantaged,Priorities
indicator,,,,
Total Inspections Since 2001,32.0,39.0,32.0,9.0


,All Tracts,Disadvantaged but Not Priorities,Not Disadvantaged,Priorities
indicator,,,,
Inspections Per Facility Since 2001,24.0,27.0,24.25,4.5


,All Tracts,Disadvantaged but Not Priorities,Not Disadvantaged,Priorities
indicator,,,,
PCT_WHITE,69.56,31.73,77.49,47.8


,All Tracts,Disadvantaged but Not Priorities,Not Disadvantaged,Priorities
indicator,,,,
Violations_PR,0.5,0.48,0.5,0.73


,All Tracts,Disadvantaged but Not Priorities,Not Disadvantaged,Priorities
indicator,,,,
Inspections_PR,0.49,0.55,0.5,0.1


In [56]:
# Show where there are significant differences between our approach and CEJST

differences = results.copy() # Make a copy of the results

# Calculate percent differences
for indicator in list(differences.index.unique()):
  differences.loc[differences.index == indicator, "All Tracts"] = round((differences.loc[differences.index == indicator, "Priorities"] / 
                                                                   differences.loc[differences.index == indicator, "All Tracts"]) * 100, 2)
  differences.loc[differences.index == indicator, "Disadvantaged but Not Priorities"] = round((differences.loc[differences.index == indicator, "Priorities"] / 
                                                                      differences.loc[differences.index == indicator, "Disadvantaged but Not Priorities"]) * 100, 2)
  differences.loc[differences.index == indicator, "Not Disadvantaged"] = round((differences.loc[differences.index == indicator, "Priorities"] / 
                                                                          differences.loc[differences.index == indicator, "Not Disadvantaged"]) * 100, 2)
  differences.loc[differences.index == indicator, "Priorities"] = round((differences.loc[differences.index == indicator, "Priorities"] / 
                                                                          differences.loc[differences.index == indicator, "Priorities"]) * 100, 2)
differences
# Output shows how much greater or less than the priority median  is for each indicator in relation to the other kinds of tracts
# >100 = Potentially disadvantaged areas have a *higher* median (e.g. a higher median population) than all US Census tracts or disadvantaged tracts or not disadvantaged tracts
# <100 = Potentially disadvantaged areas have a *lower* median (e.g. a lower median percent white population)
# NaNs and infs represent situations with 0s as denominators i.e. indicators for which the potentially disdvantaged areas have 0s as the median

# We'll first focus on indicators where priority areas have higher medians than already identified disadvantaged areas
differences[differences["Disadvantaged but Not Priorities"] > 100]

,All Tracts,Disadvantaged but Not Priorities,Not Disadvantaged,Priorities
indicator,,,,
Total population,92.49,101.92,88.73,100.0
Expected agricultural loss rate (Natural Hazards Risk Index) (percentile),162.96,628.57,133.33,100.0
Expected agricultural loss rate (Natural Hazards Risk Index),700.00,inf,233.33,100.0
Expected building loss rate (Natural Hazards Risk Index) (percentile),134.00,136.73,134.00,100.0
Expected building loss rate (Natural Hazards Risk Index),300.00,300.00,300.00,100.0
Expected population loss rate (Natural Hazards Risk Index) (percentile),130.00,132.65,132.65,100.0
Energy burden (percentile),163.27,106.67,210.53,100.0
Proximity to Risk Management Plan (RMP) facilities (percentile),132.00,103.12,150.00,100.0
Proximity to Risk Management Plan (RMP) facilities,200.00,106.06,259.26,100.0


In [57]:
# Indicators where the priorities have a *lower* median than already identified disadvantaged areas
differences[differences["Disadvantaged but Not Priorities"] < 100]

,All Tracts,Disadvantaged but Not Priorities,Not Disadvantaged,Priorities
indicator,,,,
PM2.5 in the air (percentile),106.00,94.64,112.77,100.0
PM2.5 in the air,101.06,99.08,102.01,100.0
Diesel particulate matter exposure (percentile),72.00,61.02,78.26,100.0
Diesel particulate matter exposure,74.36,63.04,80.56,100.0
Traffic proximity and volume (percentile),86.00,71.67,95.56,100.0
Traffic proximity and volume,76.52,51.02,91.36,100.0
Housing burden (percent) (percentile),136.00,87.18,183.78,100.0
Housing burden (percent),127.27,82.35,147.37,100.0
Percent pre-1960s housing (lead paint indicator) (percentile),112.24,88.71,125.00,100.0
